In [ ]:
pip install open3d numpy
!pip install mealpy

Note: you may need to restart the kernel to use updated packages.


In [33]:
import urllib.request
import zipfile
import os
from tqdm import tqdm

# URL and target paths
url = "https://modelnet.cs.princeton.edu/ModelNet40.zip"
zip_path = "ModelNet40.zip"
extract_path = "ModelNet40"

# Download with progress bar
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

print("📥 Downloading ModelNet40.zip...")
with DownloadProgressBar(unit='B', unit_scale=True, miniters=1, desc=zip_path) as t:
    urllib.request.urlretrieve(url, filename=zip_path, reporthook=t.update_to)

# Extract with progress bar
print("📦 Extracting ModelNet40.zip...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    for file in tqdm(file_list, desc="Extracting", unit="files"):
        zip_ref.extract(file, extract_path)

print("✅ Done! Files extracted to:", extract_path)


📥 Downloading ModelNet40.zip...


ModelNet40.zip: 2.04GB [07:28, 4.54MB/s]                               


📦 Extracting ModelNet40.zip...


Extracting: 100%|██████████| 12432/12432 [00:54<00:00, 227.32files/s]

✅ Done! Files extracted to: ModelNet40


In [34]:
import open3d as o3d

# Load a real object mesh (chair from ModelNet40)
mesh = o3d.io.read_triangle_mesh("ModelNet40/ModelNet40/chair/train/chair_0001.off")
mesh.compute_vertex_normals()

# Normalize: center & scale to unit size
center = mesh.get_center()
mesh.translate(-center)
bbox = mesh.get_axis_aligned_bounding_box()
scale = 1.0 / max(bbox.get_extent())
mesh.scale(scale, center=(0, 0, 0))


TriangleMesh with 2382 points and 2234 triangles.

In [35]:
import numpy as np

# --- Spherical candidates ---
def fibonacci_sphere(samples=120, radius=1):
    points = []
    phi = np.pi * (3. - np.sqrt(5.))
    for i in range(samples):
        y = 1 - (i / float(samples - 1)) * 2
        r = np.sqrt(1 - y * y)
        theta = phi * i
        x = np.cos(theta) * r
        z = np.sin(theta) * r
        points.append([x * radius, y * radius, z * radius])
    return np.array(points)

sphere_views = fibonacci_sphere(samples=120, radius=1)

# --- Surface-normal-driven candidates ---
def generate_normal_based_views(mesh, num_views=120, distance=0.3):
    pcd = mesh.sample_points_uniformly(number_of_points=num_views)
    pcd.estimate_normals()
    points = np.asarray(pcd.points)
    normals = np.asarray(pcd.normals)
    candidate_views = points - normals * distance
    return candidate_views

normal_views = generate_normal_based_views(mesh, num_views=120, distance=0.3)

# --- Combine all candidates ---
candidate_views = np.vstack([sphere_views, normal_views])
print(f"Total candidate views: {len(candidate_views)}")


Total candidate views: 240


In [36]:
from tqdm import tqdm
import open3d as o3d

pcd = mesh.sample_points_uniformly(number_of_points=5000)
view_coverage = []
for view in tqdm(candidate_views, desc="Simulating visibility"):
    _, pt_map = pcd.hidden_point_removal(view, radius=0.6)
    view_coverage.append(set(pt_map))
total_points = set(range(len(pcd.points)))


Simulating visibility: 100%|██████████| 240/240 [00:00<00:00, 256.78it/s]


In [37]:
from mealpy import FloatVar, SMA

# Parameters
init_k = 10         # SMA initial views
coverage_goal = 0.97  # Target coverage (97%)
view_budget = 60      # Max allowed views

# --- SMA for initial selection ---
def sma_objective(solution):
    indices = [int(round(x)) % len(view_coverage) for x in solution]
    indices = list(set(indices))
    covered = set()
    for i in indices:
        covered |= view_coverage[i]
    uncovered = len(total_points - covered)
    return uncovered

problem_dict = {
    "obj_func": sma_objective,
    "minmax": "min",
    "bounds": FloatVar(
        lb=(0,) * init_k,
        ub=(len(view_coverage) - 1,) * init_k,
        name="view_index"
    )
}

model = SMA.DevSMA(epoch=200, pop_size=20, p_t=0.03)
g_best = model.solve(problem_dict)

sma_selected = set(int(round(x)) % len(view_coverage) for x in g_best.solution)
covered = set().union(*[view_coverage[i] for i in sma_selected])

print(f"SMA: {len(sma_selected)} views, initial coverage = {len(covered)/len(total_points)*100:.2f}%")

# --- Greedy fill-in for remaining coverage ---
selected = set(sma_selected)
while len(selected) < view_budget and len(covered)/len(total_points) < coverage_goal:
    best_gain = 0
    best_idx = None
    for i in range(len(view_coverage)):
        if i in selected:
            continue
        gain = len(view_coverage[i] - covered)
        if gain > best_gain:
            best_gain = gain
            best_idx = i
    if best_idx is not None and best_gain > 0:
        selected.add(best_idx)
        covered |= view_coverage[best_idx]
    else:
        break

coverage_percent = len(covered) / len(total_points) * 100
print(f"\nHybrid SMA+Greedy: {len(selected)} views, coverage = {coverage_percent:.2f}%")
print(f"Selected indices: {sorted(selected)}")


2025/07/15 12:12:36 AM, INFO, mealpy.bio_based.SMA.DevSMA: DevSMA(epoch=200, pop_size=20, p_t=0.03)
2025/07/15 12:12:36 AM, INFO, mealpy.bio_based.SMA.DevSMA: >>>Problem: P, Epoch: 1, Current best: 1744.0, Global best: 1744.0, Runtime: 0.01665 seconds
2025/07/15 12:12:36 AM, INFO, mealpy.bio_based.SMA.DevSMA: >>>Problem: P, Epoch: 2, Current best: 1670.0, Global best: 1670.0, Runtime: 0.01632 seconds
2025/07/15 12:12:36 AM, INFO, mealpy.bio_based.SMA.DevSMA: >>>Problem: P, Epoch: 3, Current best: 1670.0, Global best: 1670.0, Runtime: 0.01771 seconds
2025/07/15 12:12:36 AM, INFO, mealpy.bio_based.SMA.DevSMA: >>>Problem: P, Epoch: 4, Current best: 1625.0, Global best: 1625.0, Runtime: 0.01789 seconds
2025/07/15 12:12:36 AM, INFO, mealpy.bio_based.SMA.DevSMA: >>>Problem: P, Epoch: 5, Current best: 1625.0, Global best: 1625.0, Runtime: 0.01690 seconds
2025/07/15 12:12:36 AM, INFO, mealpy.bio_based.SMA.DevSMA: >>>Problem: P, Epoch: 6, Current best: 1625.0, Global best: 1625.0, Runtime: 0.01

SMA: 10 views, initial coverage = 72.02%

Hybrid SMA+Greedy: 47 views, coverage = 83.90%
Selected indices: [0, 32, 62, 90, 102, 125, 128, 131, 134, 135, 137, 141, 142, 145, 147, 150, 151, 152, 153, 154, 159, 161, 170, 175, 178, 180, 182, 184, 185, 186, 188, 189, 193, 199, 201, 204, 212, 215, 218, 222, 224, 226, 229, 231, 232, 238, 239]


In [ ]:
spheres = []
for i, pos in enumerate(candidate_views):
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.02)
    sphere.translate(pos)
    color = [1, 0, 0] if i in selected else [0.7, 0.7, 0.7]
    sphere.paint_uniform_color(color)
    spheres.append(sphere)

o3d.visualization.draw_geometries([mesh] + spheres)